# Create a dataset

In [5]:
import sys
import os

from binconvfm.utils.download.gift_eval import list_arrow_files, PostProcessingDataset
from torch.utils.data import Dataset, DataLoader

In [6]:
dataset_name = "Salesforce/GiftEvalPretrain"
files_to_process = 4
num_workers = 4

pretrain_file_names = list_arrow_files("Salesforce/GiftEvalPretrain")
ds = PostProcessingDataset(file_names=pretrain_file_names[:files_to_process])

dataloader = DataLoader(ds, shuffle=False, batch_size=None, num_workers=num_workers)

# Define a model

In [7]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import pytorch_lightning as pl
from torch import nn

class LinearRegressionModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(32, 1)

    def forward(self, x):
        # x: [batch_size, 32, 1] => squeeze last dim to [batch_size, 32]
        x = x.squeeze(-1)
        return self.linear(x)  # Output shape: [batch_size, 1]

    def training_step(self, batch, batch_idx):
        x, y = batch
        x = x.float()
        y = y.float()
        y = y.squeeze(-1)  # shape: [batch_size, 1]
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

# Train a model

In [ ]:
model = LinearRegressionModel()
trainer = pl.Trainer(max_epochs=1, accelerator="auto")
trainer.fit(model, dataloader)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name   | Type   | Params | Mode 
------------------------------------------
0 | linear | Linear | 33     | train
------------------------------------------
33        Trainable params
0         Non-trainable params
33        Total params
0.000     Total estimated model params size (MB)
1         Modules in train mode
0         Modules in eval mode


Training: |                                                                                                   …

# Test that a model predicts something

In [ ]:
test_loader = DataLoader(ds, shuffle=False, batch_size=None)

model.eval()

for x, y in test_loader:
    with torch.no_grad():
        x = x.float()
        y_pred = model(x)
    print("Input:", x.shape)
    print("Target:", y.shape)
    break  # Only test one batch